In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook, we show how a **DFA** (Deterministic Finite Automaton) can be minimized by identifying states that are equivalent.
We will strictly adhere to the type definitions established in the previous notebooks.

## Data Structures

We import our strict types. Since `key` is defined generically over `Value` (from `recursive-set`), it can handle both original states and minimized states (sets of states) without modification.

In [ ]:
import { RecursiveSet, RecursiveMap, Tuple, Value } from "recursive-set";
import { DFA, DFAState, Char, TransRelDet } from "./01-NFA-2-DFA";

## Type Definitions

We define the types for the **Quotient Automaton** (the minimized DFA).

* **`MinState`**: A state in the minimized DFA (Level 3: Set of equivalent `DFAState`s).
* **`Pair`**: A tuple $(p, q)$ of two `DFAState`s.
* **`MinDFA`**: The structure of the minimized automaton.

In [ ]:
type MinState = RecursiveSet<DFAState>; 

// Minimized Transition Relation uses MinState as Key/Value
// Key is Tuple<[MinState, Char]>
type MinTransRel = RecursiveMap<Tuple<[MinState, Char]>, MinState>;

type Pair = Tuple<[DFAState, DFAState]>;

type MinDFA = {
    Q: RecursiveSet<MinState>;
    Σ: RecursiveSet<Char>;
    δ: MinTransRel;
    q0: MinState;
    A: RecursiveSet<MinState>;
}

### Helper Functions

**Arbitrary Element:**
The function `arb(M)` returns an *arbitrary* element from a set `M`.
In our implementation using `recursive-set`, elements are internally sorted by their **hash code** for efficiency. Therefore, this function deterministically returns the element with the smallest hash. While this is not necessarily the "smallest" value (numerically or lexicographically), it ensures a consistent canonical representative for any given set.

**Cartesian Product:**
The function `cartProd(A, B)` computes the Cartesian product $A \times B$. We leverage the built-in, optimized `cartesianProduct` method.

In [ ]:
function arb<T extends Value>(M: RecursiveSet<T>): T {
    for (const e of M) return e;
    throw new Error("Unreachable");
}

function cartProd<S extends Value, T extends Value>(
    A: RecursiveSet<S>,
    B: RecursiveSet<T>,
): RecursiveSet<Tuple<[S, T]>> {
    return A.cartesianProduct(B);
}

### Separation Logic

The function `separate` takes four arguments:
- `Pairs`: A set of pairs known to be separable.
- `States`: The set of all states $Q$.
- `Σ`: The alphabet $\Sigma$.
- `δ`: The transition map.

It returns a **new** set of separable pairs. Two states $q_1, q_2$ become separable if there exists a character $c$ such that their targets $p_1 = \delta(q_1, c)$ and $p_2 = \delta(q_2, c)$ are already known to be separable.

In [ ]:
function separate(
    Pairs: RecursiveSet<Pair>,
    States: RecursiveSet<DFAState>,
    Σ: RecursiveSet<Char>,
    δ: TransRelDet,
): RecursiveSet<Pair> {
    const newPairs = new RecursiveSet<Pair>();
        
    for (const q1 of States) {
        for (const q2 of States) {
            for (const c of Σ) {
                const p1 = δ.get(new Tuple(q1, c));
                const p2 = δ.get(new Tuple(q2, c));
                
                if (p1 && p2) {
                    const targetPair = new Tuple(p1, p2);
                    if (Pairs.has(targetPair)) {
                        newPairs.add(new Tuple(q1, q2));
                        break;
                    }
                }
            }
        }
    }
    return newPairs;
}

### Equivalence Class Lookup

Given a state `p` and a partition of the states (a set of sets), this function returns the specific set (equivalence class) that contains `p`.

In [ ]:
function findEquivalenceClass(
    p: DFAState,
    Partition: RecursiveSet<MinState>,
): MinState {
    for (const C of Partition) {
        if (C.has(p)) return C;
    }
    throw new Error(`State ${p} not found in partition`);
}

### Computing All Separable Pairs

The function `allSeparable` computes the set of all pairs $(p, q)$ that are **distinguishable**.

It uses a **fixed-point iteration**:
1.  **Base Case:** Start with pairs where one state is accepting and the other is not: $(Q \setminus A) \times A \cup A \times (Q \setminus A)$.
2.  **Iteration:** Repeatedly apply `separate` to find pairs that lead to already separated pairs. Stop when no new pairs are added.

In [ ]:
function allSeparable(
    Q: RecursiveSet<DFAState>,
    A: RecursiveSet<DFAState>,
    Σ: RecursiveSet<Char>,
    δ: TransRelDet,
): RecursiveSet<Pair> {
    const NonAccepting = Q.difference(A);
    const set1 = cartProd(NonAccepting, A);
    const set2 = cartProd(A, NonAccepting);
    let Separable = set1.union(set2);

    while (true) {
        const NewPairs = separate(Separable, Q, Σ, δ);
        if (NewPairs.isSubset(Separable)) return Separable;
        Separable = Separable.union(NewPairs);
    }
}

### Reachability Analysis

The function `reachable` filters out states that cannot be reached from the start state $q_0$. Unreachable states are irrelevant for the language accepted by the DFA and must be removed before minimization.

We provide two implementations to illustrate the difference between the algorithmic and the mathematical perspective:

The implementation directly mirrors the set-theoretical definition found in textbooks. It computes the set of reachable states as a **Fixed-Point Iteration**:
    * Start with $R_0 = \{q_0\}$.
    * Iteratively add all direct successors: $R_{i+1} = R_i \cup \{ \delta(q, c) \mid q \in R_i, c \in \Sigma \}$.
    * Stop when the set stabilizes (i.e., $R_{i+1} \subseteq R_i$).

In [ ]:
function reachable(
    q0: DFAState,
    Σ: RecursiveSet<Char>,
    δ: TransRelDet,
): RecursiveSet<DFAState> {
    let Result = new RecursiveSet<DFAState>(q0);
    while (true) {
        const NewStates = new RecursiveSet<DFAState>();
        for (const p of Result) {
            for (const c of Σ) {
                const target = δ.get(new Tuple(p, c));
                if (target) NewStates.add(target);
            }
        }
        if (NewStates.isSubset(Result)) return Result;
        Result = Result.union(NewStates);
    }
}

### The Minimization Algorithm

The `minimize` function executes the complete minimization pipeline. It constructs the **Quotient Automaton** $A / \approx$, where the states are the equivalence classes computed in the previous steps.

The algorithm proceeds as follows:

1.  **Reachability Analysis:**
    Filter out states not reachable from $q_0$, as they do not affect the accepted language.

2.  **Compute Distinguishability:**
    Calculate the set of all separable pairs using the fixed-point iteration `allSeparable`.

3.  **Group Equivalence Classes:**
    Construct the partition of the state space. Two states $p, q$ are grouped into the same class (set) if and only if they are **not** separable.
    $$[q] = \{ p \in Q \mid p \approx q \}$$

4.  **Construct the Quotient Automaton:**
    * **Start State:** The class $[q_0]$ containing the original start state.
    * **Accepting States:** The set of classes $\{ [q] \mid q \in A \}$.
    * **Transitions:** The transition function is defined as $\delta'([q], c) = [\delta(q, c)]$.

**Implementation Note:**
We determine the target state $[\delta(q, c)]$ by searching for the equivalence class that contains the successor state $\delta(q, c)$. Ideally, the minimized automaton is constructed using the generic `key` function, treating the equivalence classes themselves as the new state identifiers.

In [ ]:
function minimize(F: DFA): MinDFA {
    let { Q, Σ, δ, q0, A } = F;

    // 1. Reachability
    Q = reachable(q0, Σ, δ);
    
    // Filter Accepting states to only include reachable ones
    const reachableA = new RecursiveSet<DFAState>();
    for (const q of Q) {
        if (A.has(q)) reachableA.add(q);
    }

    // 2. Distinguishability
    const Separable = allSeparable(Q, reachableA, Σ, δ);

    // 3. Equivalence Classes
    const AllPairs = cartProd(Q, Q);
    const Equivalent = AllPairs.difference(Separable);

    const EquivClasses = new RecursiveSet<MinState>();
    for (const q of Q) {
        const classForQ = new RecursiveSet<DFAState>();
        for (const p of Q) {
            // Check equivalence (p ~ q)
            const pair = new Tuple(p, q);
            if (Equivalent.has(pair)) {
                classForQ.add(p);
            }
        }
        EquivClasses.add(classForQ);
    }

    // 4. Start State
    let newQ0: MinState | undefined;
    for (const M of EquivClasses) {
        if (M.has(q0)) {
            newQ0 = M;
            break;
        }
    }
    if (!newQ0) throw new Error("Start state vanished!");

    // 5. Accepting States
    const newAcceptSet = new RecursiveSet<MinState>();
    for (const M of EquivClasses) {
        const rep = arb(M);
        if (reachableA.has(rep)) newAcceptSet.add(M);
    }

    // 6. Transitions (Using Tuple keys for RecursiveMap)
    const newDelta = new RecursiveMap<Tuple<[MinState, Char]>, MinState>();

    for (const q of Q) {
        const classOfQ = findEquivalenceClass(q, EquivClasses);

        for (const c of Σ) {
            const p = δ.get(new Tuple(q, c));

            if (p) {
                const classOfP = findEquivalenceClass(p, EquivClasses);
                newDelta.set(new Tuple(classOfQ, c), classOfP);
            }
        }
    }

    return {
        Q: EquivClasses,
        Σ: Σ,
        δ: newDelta,
        q0: newQ0,
        A: newAcceptSet,
    };
}

The notebook `08-Test-Minimize.ipynb` provides a test for the function `minimize`.